Checking Projects with testthat
-----------------------------

In this notebook, a testthat reporter sends test output to the jupyter client.
To see this in action, paste this javascript into your console:

```
Jupyter.notebook.kernel.comm_manager.register_target('dc_project',
    function(comm, msg) {
        // comm is the frontend comm instance
        // msg is the comm_open message, which can carry data

        // Register handlers for later messages:
        comm.on_msg(function(msg) { console.log('msg:', msg)} );
        comm.on_close(function(msg) { console.log('close msg:', msg)});
        //comm.send({'foo': 0});
    })
```

In [ ]:
library(testthat)

ProjectReporter <- R6::R6Class("ProjectReporter", inherit = testthat::ListReporter,
  public = list(
    all_tests = NULL,
    comm = NULL,
    initialize = function(..., comm) {
      super$initialize(...)
      self$comm <- comm
      self$all_tests = testthat:::Stack$new()
    },
    get_results = function() {
        test_res <- lapply(self$results$as_list(),
                           self$dump_test)
        test_df <- data.frame(do.call(rbind, test_res))
        
        # summarize number of tests, etc..
        summary <- list(
                tests = length(test_res),
                failures = sum(test_df$outcome == 'fail'),
                errors = sum(test_df$outcome == 'error')
                )

        payload <- list(
                success = all(as.logical(test_df$success)),
                summary = summary,
                tests = test_res
                )
        
        self$comm$send(payload)

        payload
    },
    dump_test = function(test) {
        message <- paste(
            lapply(test$results, `[[`, 'message'),
            collapse = '\n')

        res <- testthat:::sumarize_one_test_results(test)
        success <- !any(res$failed, res$error)
        # figure out outcome, e.g. for counting errors later
        if (!success) {
            outcome <- if (res$failed) 'fail' else 'error'
        } else outcome <- 'success'

        list(name = test$test,
             message = message,
             success = !any(res$failed, res$error),
             outcome = outcome)
    }
  ),
  private = list(
  )
)

.run_tests <- function(test_expr) {
    comm <- IRkernel::comm_manager()$new_comm('dc_project')
    comm$open()

    reporter <- ProjectReporter$new(comm = comm)
    reporter$start_file('some name')
    env = test_env()
    tests <- substitute(test_expr)
    with_reporter(
        reporter = reporter, start_end_reporter = TRUE,
        eval(tests, envir = env)
        )
    reporter
}

In [ ]:
out <- .run_tests({
    test_that("a passing test", {
        expect_equal(2, 2)
        expect_equal(3, 3)
    })

    test_that("a failing test", {
        expect_equal(2, 3)
        expect_equal(3, 4)
    })
    
    test_that("an erroring test", {
        stopifnot(FALSE)
        expect_equat(2, 2)
    })
})
out$get_results()